bài 1:

In [2]:
def generate_pearson_sql(table_name: str, col_a: str, col_b: str) -> str:
    sql = f"""
    SELECT
      (
        COUNT(*) * SUM({col_a} * {col_b}) - SUM({col_a}) * SUM({col_b})
      ) /
      (
        SQRT(COUNT(*) * SUM({col_a} * {col_a}) - POWER(SUM({col_a}), 2)) *
        SQRT(COUNT(*) * SUM({col_b} * {col_b}) - POWER(SUM({col_b}), 2))
      ) AS pearson_correlation
    FROM {table_name};
    """
    return sql.strip()

# Ví dụ dùng:
table = "data_table"
col_a = "a"
col_b = "b"

print(generate_pearson_sql(table, col_a, col_b))


SELECT
      (
        COUNT(*) * SUM(a * b) - SUM(a) * SUM(b)
      ) /
      (
        SQRT(COUNT(*) * SUM(a * a) - POWER(SUM(a), 2)) *
        SQRT(COUNT(*) * SUM(b * b) - POWER(SUM(b), 2))
      ) AS pearson_correlation
    FROM data_table;


bài 2

In [7]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# ===== BƯỚC 1: Tạo cơ sở dữ liệu SQLite trong bộ nhớ =====
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# ===== BƯỚC 2: Tạo bảng dữ liệu =====
cursor.execute('''
CREATE TABLE mau_xe (
    day INTEGER,
    model_A FLOAT,
    model_B FLOAT,
    model_C FLOAT
)
''')

# ===== BƯỚC 3: Nhập dữ liệu =====
data = [
    (1, 8, 9, 7),
    (2, 7.5, 8.5, 7),
    (3, 6, 7, 8),
    (4, 7, 6, 5)
]
cursor.executemany('INSERT INTO mau_xe (day, model_A, model_B, model_C) VALUES (?, ?, ?, ?)', data)
conn.commit()

# ===== BƯỚC 4: Lấy dữ liệu ra dạng DataFrame =====
df = pd.read_sql_query('SELECT * FROM mau_xe', conn)
print("Dữ liệu gốc:")
print(df)

# ===== BƯỚC 5: Chuyển dữ liệu sang dạng quan hệ (long format) =====
df_lg = pd.melt(df, id_vars=['day'], var_name='model', value_name='score')
print("\nDữ liệu dạng quan hệ:")
print(df_lg)

# ===== BƯỚC 6: Tạo cột chứng nhận (Certified) theo ngưỡng ≥ 7 =====
df_lg['certified'] = df_lg['score'].apply(lambda x: 1 if x >= 7 else 0)

# ===== BƯỚC 7: Kiểm định χ² theo mẫu xe =====
ct_model = pd.crosstab(df_lg['model'], df_lg['certified'])
chi2_model, p_model, dof_model, _ = chi2_contingency(ct_model)

print("\n=== Kiểm định theo mẫu xe ===")
print(ct_model)
print(f"Chi2 = {chi2_model:.2f}, p-value = {p_model:.4f}, dof = {dof_model}")
if p_model < 0.05:
    print("=> Có sự khác biệt đáng kể giữa các mẫu xe.")
else:
    print("=> Không có sự khác biệt đáng kể giữa các mẫu xe.")

# ===== BƯỚC 8: Kiểm định χ² theo ngày =====
ct_day = pd.crosstab(df_lg['day'], df_lg['certified'])
chi2_day, p_day, dof_day, _ = chi2_contingency(ct_day)

print("\n=== Kiểm định theo ngày ===")
print(ct_day)
print(f"Chi2 = {chi2_day:.2f}, p-value = {p_day:.4f}, dof = {dof_day}")
if p_day < 0.05:
    print("=> Kết quả chứng nhận phụ thuộc vào ngày thử nghiệm.")
else:
    print("=> Kết quả chứng nhận không phụ thuộc vào ngày thử nghiệm.")


Dữ liệu gốc:
   day  model_A  model_B  model_C
0    1      8.0      9.0      7.0
1    2      7.5      8.5      7.0
2    3      6.0      7.0      8.0
3    4      7.0      6.0      5.0

Dữ liệu dạng quan hệ:
    day    model  score
0     1  model_A    8.0
1     2  model_A    7.5
2     3  model_A    6.0
3     4  model_A    7.0
4     1  model_B    9.0
5     2  model_B    8.5
6     3  model_B    7.0
7     4  model_B    6.0
8     1  model_C    7.0
9     2  model_C    7.0
10    3  model_C    8.0
11    4  model_C    5.0

=== Kiểm định theo mẫu xe ===
certified  0  1
model          
model_A    1  3
model_B    1  3
model_C    1  3
Chi2 = 0.00, p-value = 1.0000, dof = 2
=> Không có sự khác biệt đáng kể giữa các mẫu xe.

=== Kiểm định theo ngày ===
certified  0  1
day            
1          0  3
2          0  3
3          1  2
4          2  1
Chi2 = 4.89, p-value = 0.1801, dof = 3
=> Kết quả chứng nhận không phụ thuộc vào ngày thử nghiệm.


bài 3

In [10]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng flights với cột departure_time
cursor.execute('''
CREATE TABLE flights (
    departure_time INTEGER
)
''')

# Nhập dữ liệu vào bảng
data = [
    (831,),
    (1425,),
    (104,),
    (24,),
    (0,),
    (1230,),
    (19,),
    (2349,)
]
cursor.executemany('INSERT INTO flights (departure_time) VALUES (?)', data)
conn.commit()

# Truy vấn dữ liệu từ bảng flights
df = pd.read_sql_query('SELECT * FROM flights', conn)
print("== Dữ liệu ban đầu ==")
print(df)

# Hàm chuyển đổi số nguyên sang định dạng 12h (AM/PM)
def convert_to_time_format(departure_time):
    hour = departure_time // 100
    minute = departure_time % 100
    time_str = f"{hour:02}:{minute:02}"
    return pd.to_datetime(time_str, format="%H:%M").strftime('%I:%M %p')

# Áp dụng chuyển đổi vào DataFrame
df['departure_time_converted'] = df['departure_time'].apply(convert_to_time_format)

# In kết quả sau khi chuyển đổi
print("\n== Dữ liệu sau khi chuyển đổi thời gian ==")
print(df)


== Dữ liệu ban đầu ==
   departure_time
0             831
1            1425
2             104
3              24
4               0
5            1230
6              19
7            2349

== Dữ liệu sau khi chuyển đổi thời gian ==
   departure_time departure_time_converted
0             831                 08:31 AM
1            1425                 02:25 PM
2             104                 01:04 AM
3              24                 12:24 AM
4               0                 12:00 AM
5            1230                 12:30 PM
6              19                 12:19 AM
7            2349                 11:49 PM


bài 4

In [11]:
import pandas as pd
import numpy as np

# Dữ liệu điểm ví dụ
data = {
    'student_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'name': ['Vy', 'Bình', 'Lục', 'Dũng', 'Vinh', 'Hoàng', 'My', 'Mai', 'Nam', 'Oanh', 'Phúc', 'Quang'],
    'score': [9.2, 9.2, 7.1, 9.9, 8.8, 9.5, 10.0, 7.0, 6.6, 9.2, 8.5, 9.8]
}
df = pd.DataFrame(data)

# Tính median
median_score = df['score'].median()

# Tính độ lệch tuyệt đối so với median
df['deviation'] = abs(df['score'] - median_score)

# Tính MAD
mad = df['deviation'].median()

# Đánh dấu giá trị ngoại lệ nếu độ lệch > 1.5 * MAD
df['is_outlier'] = df['deviation'] > 1.5 * mad

# Hiển thị kết quả
print("== Phát hiện ngoại lệ theo MAD ==")
print(df[['student_id', 'name', 'score', 'deviation', 'is_outlier']])


== Phát hiện ngoại lệ theo MAD ==
    student_id   name  score  deviation  is_outlier
0            1     Vy    9.2        0.0       False
1            2   Bình    9.2        0.0       False
2            3    Lục    7.1        2.1        True
3            4   Dũng    9.9        0.7       False
4            5   Vinh    8.8        0.4       False
5            6  Hoàng    9.5        0.3       False
6            7     My   10.0        0.8       False
7            8    Mai    7.0        2.2        True
8            9    Nam    6.6        2.6        True
9           10   Oanh    9.2        0.0       False
10          11   Phúc    8.5        0.7       False
11          12  Quang    9.8        0.6       False


bài 5

In [12]:
import pandas as pd
from itertools import combinations

# Dữ liệu gốc từ bảng Patient
data = [
    {'id': 1, 'last_name': 'Le', 'weight': 70, 'height': 175},
    {'id': 2, 'last_name': 'Tran', 'weight': 65, 'height': 160},
    {'id': 3, 'last_name': 'Luu', 'weight': 60, 'height': 170},
    {'id': 4, 'last_name': 'Nguyen', 'weight': 70, 'height': 175},
    {'id': 5, 'last_name': 'Le', 'weight': 55, 'height': 160},
]

df = pd.DataFrame(data)

# Tạo các cặp bệnh nhân
pairs = list(combinations(df.to_dict('records'), 2))

# So sánh khoảng cách Boolean trên last_name và weight
results = []
for p1, p2 in pairs:
    bool_last_name = 0 if p1['last_name'] == p2['last_name'] else 1
    bool_weight = 0 if p1['weight'] == p2['weight'] else 1
    boolean_distance = bool_last_name + bool_weight

    # Gợi ý kết luận
    if boolean_distance == 0:
        conclusion = 'Có thể là cùng người'
    elif boolean_distance == 1:
        conclusion = 'Khá giống nhau'
    else:
        conclusion = 'Không phải cùng người'

    results.append({
        'id_1': p1['id'],
        'id_2': p2['id'],
        'last_name_1': p1['last_name'],
        'last_name_2': p2['last_name'],
        'weight_1': p1['weight'],
        'weight_2': p2['weight'],
        'boolean_distance': boolean_distance,
        'conclusion': conclusion
    })

# Hiển thị kết quả
df_result = pd.DataFrame(results)
print("== Kết quả so sánh bằng khoảng cách Boolean ==")
print(df_result)


== Kết quả so sánh bằng khoảng cách Boolean ==
   id_1  id_2 last_name_1 last_name_2  weight_1  weight_2  boolean_distance  \
0     1     2          Le        Tran        70        65                 2   
1     1     3          Le         Luu        70        60                 2   
2     1     4          Le      Nguyen        70        70                 1   
3     1     5          Le          Le        70        55                 1   
4     2     3        Tran         Luu        65        60                 2   
5     2     4        Tran      Nguyen        65        70                 2   
6     2     5        Tran          Le        65        55                 2   
7     3     4         Luu      Nguyen        60        70                 2   
8     3     5         Luu          Le        60        55                 2   
9     4     5      Nguyen          Le        70        55                 2   

              conclusion  
0  Không phải cùng người  
1  Không phải cùng người  
2 